In [110]:
import torch
import config
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as TF

from enum import Enum
from datetime import datetime
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

In [111]:
class Modes(Enum):
    TRAIN = 'train'
    TEST = 'test'

In [112]:
def getDataFromCSV(mode, file, start, end):
    return pd.read_csv(f'{config.samples_path}\\{mode}\\{file}',
                               usecols = range(start, end))


In [113]:
def normallizeData(data):
    std_history = []
    mean_history = []

    for i in range(data.shape[1]):
        # Compute standard deviation
        std = torch.std(data[:, i])
        std = 0.001 if std == 0 else std

        # Save the std and mean history for denormalize later
        std_history.append(std)
        mean_history.append(torch.mean(data[:, i]))

        # Normalize the data
        data[:, i] = (data[:, i] - torch.mean(data[:, i])) / std
        
    return data, std_history, mean_history

In [114]:
def getData(mode):
    # Get data from csv files
    x = getDataFromCSV(mode, config.x_data_file, 
                        config.x_cols_start_index, config.x_cols_end_index)
                        
    y = getDataFromCSV(mode, config.y_data_file, 
                        config.y_cols_start_index, config.y_cols_end_index)

    # Transforms the data to tensors
    x_tensor = torch.tensor(x.values, requires_grad=True).float()
    y_tensor = torch.tensor(y.values, requires_grad=True).float()

    # Normallize the data
    norm_x_tensor, _, _ = normallizeData(x_tensor)
    norm_y_tensor, y_std_history, y_mean_history = normallizeData(y_tensor)

    return norm_x_tensor, norm_y_tensor, y_std_history, y_mean_history

In [115]:
class LandmarksDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y


    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return len(self.x)

In [116]:
def getDataset(mode):
    x, y, y_std_history, y_mean_history = getData(mode)
    return LandmarksDataset(x, y), y_std_history, y_mean_history

In [117]:
class Net(nn.Module):
    def __init__(self, in_num_of_features, out_num_of_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_num_of_features, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, out_num_of_features)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x


In [118]:
def train(epochs, train_loader, net, optimizer, criterion, log_interval):
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
                
    return net

In [119]:
def test(test_loader, net, criterion):
    test_loss = 0
    # correct = 0
    results = torch.tensor([]).float()

    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        net_out = net(data)
        results = torch.cat((results, net_out))
        # sum up batch loss
        test_loss += criterion(net_out, target).item()
        # pred = net_out.data.max(1)[1]  # get the index of the max log-probability
        # correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}'.format(
        test_loss))

    return results, test_loss

In [120]:
def getFeatureSize(data, target):
    # Convert tensors to numpy
    data, target = data.detach().numpy(), target.detach().numpy()
    return data.shape[0], target.shape[0]

In [121]:
def denormalizeData(data, std_history, mean_history):
    for i in range(data.shape[1]):
        std, mean = std_history[i], mean_history[i]
        data[:, i] = (data[:, i] + mean) * std
    
    return data

In [122]:
def saveResultsToOutputFile(mode, filename, results):
    # Create the output csv columns
    blend_cols = []
    quat_cols = ['Quaternion_x', 'Quaternion_y', 'Quaternion_z', 'Quaternion_w']
    for i in range (config.blend_range[0], config.blend_range[1]):
        blend_cols.append('Blendshape_{0}'.format(i))
    output_cols = quat_cols + blend_cols
    
    # Convert the results to data frame 
    results = results.detach().numpy()
    results = pd.DataFrame(results, columns=output_cols)
    
    # Assign values outside boundary to boundary values
    results.loc[:, quat_cols] = results.loc[:, quat_cols].clip(config.quat_domain[0], config.quat_domain[1])
    results.loc[:, blend_cols] = results.loc[:, blend_cols].clip(config.blend_domain[0], config.blend_domain[1])

    # Save the results to the output csv file
    file_path = f'{config.samples_path}\\{mode}\\{filename}'
    results.to_csv(file_path)
    print(f'[{datetime.now()}] Successfully saved the results to {file_path}')

In [123]:
def create_nn(batch_size=50, learning_rate=0.001, epochs=20,
              log_interval=10):
    
    # Create the train data loader
    train_dataset, _, _ = getDataset(Modes.TRAIN.value)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size,
        shuffle=True)

    # Create the test data loader
    test_dataset, std_history, mean_history = getDataset(Modes.TEST.value)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size,
        shuffle=False)

    # Extract the input and output num of features
    in_num_of_features, out_num_of_features = getFeatureSize(*train_dataset[0])

    # Create the net
    net = Net(in_num_of_features, out_num_of_features)

    # Create an optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # Create a loss function
    criterion = nn.MSELoss(reduction='mean')
    # criterion = nn.L1Loss(reduction='mean')

    # Run the main training loop
    net = train(epochs, train_loader, net, optimizer, criterion, log_interval)
                
    # Run a test loop
    results, test_loss = test(test_loader, net, criterion)
    
    # De-normalize the data to the original domains
    results = denormalizeData(results, std_history, mean_history)
    
    # Save the results to output file
    saveResultsToOutputFile(Modes.TEST.value, config.output_filename, results)

In [124]:
if __name__ == "__main__":
    create_nn()

Train Epoch: 0 [0/3269 (0%)]	Loss: 1.021283
Train Epoch: 0 [500/3269 (15%)]	Loss: 0.971901
Train Epoch: 0 [1000/3269 (30%)]	Loss: 0.922604
Train Epoch: 0 [1500/3269 (45%)]	Loss: 0.960005
Train Epoch: 0 [2000/3269 (61%)]	Loss: 0.895052
Train Epoch: 0 [2500/3269 (76%)]	Loss: 0.982850
Train Epoch: 0 [3000/3269 (91%)]	Loss: 0.984340
Train Epoch: 1 [0/3269 (0%)]	Loss: 0.984140
Train Epoch: 1 [500/3269 (15%)]	Loss: 0.828951
Train Epoch: 1 [1000/3269 (30%)]	Loss: 0.781886
Train Epoch: 1 [1500/3269 (45%)]	Loss: 0.817600
Train Epoch: 1 [2000/3269 (61%)]	Loss: 0.890351
Train Epoch: 1 [2500/3269 (76%)]	Loss: 0.866007
Train Epoch: 1 [3000/3269 (91%)]	Loss: 0.725065
Train Epoch: 2 [0/3269 (0%)]	Loss: 0.675849
Train Epoch: 2 [500/3269 (15%)]	Loss: 0.653198
Train Epoch: 2 [1000/3269 (30%)]	Loss: 0.737489
Train Epoch: 2 [1500/3269 (45%)]	Loss: 0.665914
Train Epoch: 2 [2000/3269 (61%)]	Loss: 0.638704
Train Epoch: 2 [2500/3269 (76%)]	Loss: 0.677700
Train Epoch: 2 [3000/3269 (91%)]	Loss: 0.698022
Train E